In [1]:
import sys   ###qmdp1（上の2つのセル）
sys.path.append('../scripts/')
from dp_policy_agent import *

In [2]:
class QmdpAgent(DpPolicyAgent):
    def __init__(self, time_interval, estimator, goal, puddles, sampling_num=10, widths=np.array([0.2, 0.2, math.pi/18]).T, \
                 puddle_coef=100.0, lowerleft=np.array([-4, -4]).T, upperright=np.array([4, 4]).T): 
        super().__init__(time_interval, estimator, goal, puddle_coef, widths, lowerleft, upperright)
        
    def normalize1(self, t): #正規化方法1（-π〜π）
            while t < -np.pi: t += 2*np.pi
            while t >= np.pi: t -= 2*np.pi
            return t
        
    def normalize2(self, t): #正規化方法1（0〜2π）
            while t < 0.0: t += 2*np.pi
            while t >= 2*np.pi: t -= 2*np.pi
            return t
        
    def avg(self, particles):
        x = np.array([p.pose[0] for p in particles]).mean()
        y = np.array([p.pose[1] for p in particles]).mean()
        
        ts1 = np.array([self.normalize1(p.pose[2]) for p in particles])
        ts2 = np.array([self.normalize2(p.pose[2]) for p in particles])
        
        t = ts1.mean() if ts1.var() < ts2.var() else ts2.mean()
        
        return np.array([x,y,t]).T
        
    def policy(self, pose, goal=None): #平均値を返す
        pose = self.avg(self.estimator.particles)
        return self.policy_data[self.to_index(pose, self.pose_min, self.index_nums, self.widths)]

In [3]:
def trial(animation):  ###qmdp1exec
    time_interval = 0.1
    world = PuddleWorld(30, time_interval, debug=not animation) 

    ##ランドマークの追加（意地悪な位置に）##
    m = Map()
    m.append_landmark(Landmark(1,4))
    m.append_landmark(Landmark(4,1))
    m.append_landmark(Landmark(-4,-4))
    world.append(m)
    
    ##ゴール・水たまりの追加（これは特に変更なし）##
    goal = Goal(-3,-3)
    puddles = [Puddle((-2, 0), (0, 2), 0.1), Puddle((-0.5, -2), (2.5, 1), 0.1)] 
    world.append(goal)
    world.append(puddles[0]) 
    world.append(puddles[1])
    
    ##ロボットを作る##
    init_pose = np.array([2.5, 2.5, 0]).T
    pf = Mcl(m, init_pose, 100)
    a = QmdpAgent(time_interval, pf, goal, puddles)
    r = Robot(init_pose, sensor=Camera(m), agent=a, color="red")

    world.append(r)
        
    world.draw()
    
    return a.total_reward+a.final_value, a.in_goal

In [ ]:
trial(True)

In [4]:
total = 0.0
success = 0
num = 100
for i in range(num):
    reward, goal = trial(False)
    print(reward, goal)
    if goal: success += 1
    total += reward
    
print(total/num, success)

-12.399999999999972 True
-25.900000000000066 True
-11.299999999999976 True
-11.199999999999976 True
-11.899999999999974 True
-11.799999999999974 True
-12.199999999999976 True
-11.599999999999975 True
-12.299999999999972 True
-13.09999999999997 True
-12.399999999999972 True
-12.599999999999971 True
-58.2000000000001 True
-13.199999999999973 True
-14.999999999999963 True
-40.00000000000008 True
-68.69999999999986 True
-13.799999999999967 True
-13.599999999999968 True
-14.299999999999965 True


/usr/local/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


-28.600000000000083 True
-24.00000000000009 True
-13.799999999999967 True
-11.999999999999973 True
-24.300000000000068 True
-25.200000000000085 True
-22.800000000000075 True
-14.399999999999965 True
-13.799999999999967 True
-11.999999999999973 True
-26.000000000000078 True
-13.499999999999968 True
-12.499999999999972 True
-18.400000000000006 True
-12.499999999999972 True
-47.3000000000001 True
-32.40000000000007 True
-14.599999999999964 True
-49.70000000000007 True
-14.599999999999964 True
-13.799999999999967 True
-12.299999999999972 True
-26.200000000000113 True
-17.900000000000002 True
-14.199999999999969 True
-13.299999999999969 True
-63.6000000000001 True
-45.30000000000009 True
-13.299999999999969 True
-15.099999999999962 True
-18.10000000000001 True
-12.69999999999997 True
-11.599999999999975 True
-13.399999999999968 True
-11.399999999999975 True
-50.300000000000075 True
-13.199999999999969 True
-71.29999999999983 True
-19.600000000000037 True
-12.599999999999971 True
-13.7999999

In [ ]:
-23.118999999999986 98